# Setting up libraries and data

In [1]:
# install libraries
# using the environment `nam_test`
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install pytorch-lightning
!pip install torchinfo
!pip install numpy
!pip install torchmetrics
!pip install torchviz
!pip install torchvision

In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchmetrics
# from torchvision.models import resnet50
from torchvision.models.feature_extraction import create_feature_extractor
from torch.distributions import multivariate_normal

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.logging.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from functools import reduce
from operator import __add__

from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

import scipy

# SRED data downloaded from
# https://drive.google.com/file/d/1OCQiYs1183XmFIyW0_Zd5oX4yp77vrfQ/view?usp=sharing

# define data path
data_path = "./data/"

c:\Users\iazizi\.conda\envs\nam_test\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Run below to free GPU memory (in case of GPU problems)

In [3]:
# to clean cache in case of any problems
torch.cuda.empty_cache()

# Models and Architectures
Define various architectures for the neural networks (normal SRED, Supervised VAE, Marc-Olivier's idea).

## EM-VAE Simple Implementation

### Data loading

In [4]:
# function to scale the input featuers
def scale_tabular_features(csv_file, training_set = False, train_sigma = None):
    tabular = pd.read_csv(csv_file,dtype={'listing_id': 'Int64'})
    # processing the tabular information
    tabular_new = tabular[["living_space", "rooms", "lat", "lon"]].copy()
    # scaling the inputs
    # this would not work for the test set, since we will otherwise using a different mean and sd
    sc = StandardScaler()
    sc.fit(tabular_new)
    # if we do not do this, we risk using the same information for scaling the training and test sets
    if training_set:
        sc.scale_ = np.std(tabular_new, axis=0, ddof=1).to_list()
    else:
        # if we do not do this, we risk using the same information for scaling the training and test sets
        sc.scale_ = train_sigma
    tabular_new = pd.DataFrame(sc.transform(tabular_new), columns=['living_space', 'rooms', 'lat', 'lon'])
    tabular_new[["price","listing_id"]] = tabular[["price", "listing_id"]]
    if training_set:
        return[sc.scale_,tabular_new]
    else:
        return tabular_new

# class to load the tabular data
class TabularDataset(Dataset):
    """Tabular dataset."""

    def __init__(self, tabular, num_workers = int(os.cpu_count())):
        self.tabular = tabular
        self.num_workers = num_workers
        # self.tabular = pd.read_csv(self.csv_file,dtype={'listing_id': 'Int64'})

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        # to load all the data into memory, have a look here
        # this is unncessary if you have many cpus on your machine as it is the case on sagemaker
        # https://stackoverflow.com/questions/72012067/pytorch-dataloader-dataset-complete-in-ram
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        # extract the outcome
        y = tabular["price"]
        # log transform the data
        y = np.log(y)
        
        # processing the tabular information
        tabular = tabular[["living_space", "rooms", "lat", "lon"]]
        # scaling the inputs
        # this would not work for the test set, since we will otherwise using a different mean and sd
        # then we extrac the outputs as tensors
        
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return tabular, y

### Simple implementation

In [5]:
# load the data
train_scaled_tabular = scale_tabular_features(csv_file=f"{data_path}/metadata/train_data.csv", training_set = True)
scaling_stds = train_scaled_tabular[0]
train_tabular = train_scaled_tabular[1]
all_train_data = TabularDataset(tabular = train_tabular)
train_batch_load = torch.utils.data.DataLoader(dataset=all_train_data, batch_size=15)
tabular, y = next(iter(train_batch_load))
print(tabular)
# check out this when you want to expand it to larger batches
# https://stackoverflow.com/questions/57386851/how-to-get-entire-dataset-from-dataloader-in-pytorch

tensor([[-0.5213, -1.1537,  1.0005, -0.5727],
        [-0.0406,  0.9625,  0.1501, -0.3315],
        [ 0.3761, -0.0956,  0.9929, -0.2967],
        [-0.5534, -0.0956,  0.0299,  0.3526],
        [-0.7136, -0.0956, -0.5376, -0.7219],
        [ 0.6004,  0.9625, -0.6296, -0.4600],
        [ 0.6645,  0.9625,  0.1702,  0.1675],
        [-0.7777, -0.6247, -0.3913, -1.3206],
        [ 1.1132,  0.9625,  0.1348, -0.4891],
        [-1.1623, -1.6828, -0.4512,  0.4030],
        [ 0.7606, -0.0956, -0.3841, -0.4001],
        [-0.0726, -0.0956,  1.0388, -0.5113],
        [-0.8418, -0.0956, -1.1259, -1.1764],
        [ 0.1197, -0.0956,  0.6585,  1.6515],
        [-0.5854,  0.4335,  0.6585,  1.6590]])


In [6]:
# MODEL ARCHITECTURE (custom)
class SetupTabVAE(pl.LightningModule):
    def __init__(
        self, num_workers: int = int(os.cpu_count()), batch_size: int = 32, lr: float = 1e-3
    ):
        super().__init__()
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.lr = lr
        
    def setup(self, stage):
        # load the training and testing datasets and make the validation split
        train_scaled_tabular = scale_tabular_features(csv_file=f"{data_path}/metadata/train_data.csv", training_set = True)
        scaling_stds = train_scaled_tabular[0]
        train_tabular = train_scaled_tabular[1]
        all_train_data = TabularDataset(tabular = train_tabular)   
        train_size = int(0.90 * len(all_train_data))
        val_size = int((len(all_train_data) - train_size))
        self.train_set, self.val_set = random_split(all_train_data, (train_size, val_size))        
        # give the custom stds for the test set
        test_scaled_tabular = scale_tabular_features(csv_file=f"{data_path}/metadata/test_data.csv", training_set = False, train_sigma = scaling_stds)
        self.test_set = TabularDataset(tabular = test_scaled_tabular)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, num_workers = int(os.cpu_count()),pin_memory=True)

#     def val_dataloader(self):
#         return DataLoader(self.val_set, batch_size=self.batch_size, num_workers = int(os.cpu_count()),pin_memory=True)

#     def test_dataloader(self):
#         return DataLoader(self.test_set, batch_size=self.batch_size, num_workers = int(os.cpu_count()),pin_memory=True)

### Integrating EM-VAE into one model

In [ ]:
# nice blog on visualizing the activations
# https://pytorch-lightning.readthedocs.io/en/stable/notebooks/course_UvA-DL/03-initialization-and-optimization.html

In [29]:
# MODEL ARCHITECTURE (custom)
# SOME VERY NICE IDEAS HERE for multiple optimization and 
# https://pytorch-lightning.readthedocs.io/en/stable/model/manual_optimization.html

class ModelCalculateZ(SetupTabVAE):
    def __init__(
        self, lr: float = 1e-3, batch_size: int = 32, num_workers: int = int(os.cpu_count()), sigma_squared = 1
    ):
        super().__init__(batch_size = batch_size)
        self.lr = lr

        self.layer_h = nn.Linear(4, 10)
        self.layer_z = nn.Linear(10, 20)
        self.layer_j = nn.Linear(20, 10)
        self.layer_y = nn.Linear(10,1)
        
        # disable training for layers j and y (may have another network for that)
        for param in self.layer_j.parameters():
            param.requires_grad = False
        for param in self.layer_y.parameters():
            param.requires_grad = False

        # define other parameters
        self.sigma_squared = sigma_squared
        
        # define the activation function for the middle layers
        self.tanh_fun = nn.Tanh()
        
        # apply the weights
        self.apply(self._init_weights)
        
        # we would also manually optimize
        # self.automatic_optimization = False
        
    # define weight initialization (here we use Glorot uniform but can be anything)
    # https://stackoverflow.com/questions/49433936/how-do-i-initialize-weights-in-pytorch
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # torch.nn.init.xavier_uniform_(module.weight)
            module.weight.data.normal_(mean=0.0, std=1.0)
            if module.bias is not None:
                # module.bias.data.zero_()
                module.bias.data.fill_(0)
    
    def encode(self, tab):
        # defining tabular inputs
        layer_h = self.layer_h(tab)
        layer_h = self.tanh_fun(layer_h)
        z_tilde = self.layer_z(layer_h)
        # not sure about this step but can try with and without
        z_tilde = self.tanh_fun(z_tilde)
        # not sure if this should be with activation function or without
        return z_tilde

    # function to simulate z -> The N has to be changed to the batch size
    # batch size is not correct as it will change for the last iteration, but for now it does the job
    
    # step 7: we do the importance sampling (monte-carlo simulation)
    # def simulate_z(self, z_tilde_output, R: int = 5):
    #     # this N should probably by something else (automatically use the batch)
    #     # parallelize this over multiple CPUs
    #     # this basically means that the length of our list stays size as the batch size and then we next inside
    #     z_tilde_output = z_tilde_output.detach().numpy()
    #     return z_tilde_output
    #     z_means = []
    #     for r in range(R):
    #         z_means.append(torch.from_numpy(np.random.normal(z_tilde_output)).float())
    #     return z_means
    
    # this could be of some help as well
    # https://discuss.pytorch.org/t/is-there-a-way-to-apply-an-operation-to-a-single-sample-in-a-batch/67135/8
    # https://discuss.pytorch.org/t/applying-a-function-on-each-individual-element-of-a-tensor/143621/2
    def simulate_z(self, z_tilde_outputs, R: int = 5):
        N = self.batch_size
        z_means = []
        # this N should probably by something else (automatically use the batch)
        for i in range(N):
            # parallelize this over multiple CPUs
            # this basically means that the length of our list stays size as the batch size and then we next inside
            z_means.append([i])
            z_tilde_output = z_tilde_outputs[i]
            z_tilde_output = z_tilde_output.detach().numpy()
            for r in range(R):
                z_means[i].append(torch.from_numpy(np.random.normal(z_tilde_output)).float())
        return z_means

    # step 8: obtain the \tilde{y} by running `g_{\phi'}(z_{i}^{(r)})`
    ## remark: pass that through network with the weights of phi to get the g_phi(z)
    ## remark from marc-olivier: 5 times (R) times for each one you get y_tilde
    def get_y_tilde(self, simulated_z):
        y_tilde = []
        for r in range(len(simulated_z[0])): #this was just an arbitary indexing since all Rs are the same
        # for copying the weights, you can also have a look at https://discuss.pytorch.org/t/loading-a-specific-layer-from-checkpoint/52725
        # we place 1 here in the second part since the second element of the list is the tensor (first is just the `ith` element)
            new_layer_j = self.layer_j(simulated_z[r]) #this is plus 1 since the first element is the index itself
            new_layer_y = self.layer_y(new_layer_j)
            y_tilde.append(new_layer_y)
        return y_tilde

    # step 9: computing `p'(y_i|z_i^r)`
    def get_p_prime_y_z(self, y_tilde):
        p_prime_y_z = []
        for r in range(len(y_tilde[0])):
            pdf_val = torch.from_numpy(scipy.stats.norm.pdf(y[i], y_tilde[1].detach().numpy(), 1))
            p_prime_y_z.append(pdf_val)
        return p_prime_y_z

    # step 10: Compute the `w` (scaled contributions)
    ## remark from marc-o: for the division (step 10), you have 5 p primes divided by the sum of p_primes 5, basically standardize the series of 5 by one
    ## remark two from marc-O: `w` is standardization
    def get_w(self, p_prime_y_z):
        w = []
        w_r = []
        # maybe can turn the loop into a function
        denominator = sum(p_prime_y_z[0:]) #we slice to get everything but the first element
        for r in range(len(p_prime_y_z[0])):
            numerator = p_prime_y_z[r]
            # to avoid dividing zero by zero as undefined
            w_i_r = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
            w_r.append(w_i_r)
        # concatenate
        w.append(torch.from_numpy(np.concatenate(w_r[0:]).ravel()))
        return w

    # step 11: optimize for theta
    # custom loss function
    # for inspriation, checkout VAE in lightning: https://github.com/williamFalcon/pytorch-lightning-vae/blob/main/vae.py
    def get_theta_loss(self,intermediate_outputs, simulated_z, w):
        theta_loss = []
        r_sum = []
        n_sum = []
        for i in range(len(w)):
            theta_loss.append([i])
            f_theta_l_xi = intermediate_outputs[i]
            w_multiplier =  w[i][1:]
            # I do w[1] since the rs are all equal (5) so it would be like 5r x 15i
            for r in range(len(w[1])-1):
                # # not sure about the calculation of the second component of mse so ask to marc-o about it
                # squared euclean norm here https://stats.stackexchange.com/questions/181620/what-is-the-meaning-of-super-script-2-subscript-2-within-the-context-of-norms
                mse = ((simulated_z[i][r+1] - f_theta_l_xi)** 2)
                # # weighted_mse
                weighted_mse = torch.sum(mse)
                # multiply and flatten numpy
                # CHECK ABOUT THE MULTIPICATION TYPE HERE TO MAKE SURE THAT IT'S RIGHT (element wise and not a matrix multipication)
                weighted_mse = torch.ravel(torch.mul(weighted_mse,w_multiplier[0]))
                theta_loss[i].append(weighted_mse)
            # theta_loss[i].append(r_sums)

            # sum the array by R
            r_sum.append(torch.sum(theta_loss[i][1]))

        # sum by n
        n_sum = sum(r_sum)
        
        logs = {
            "theta_loss": n_sum
        }
            
        return n_sum, logs
    
    # to get a specific layer values
    # get the forward hook intisde the function
    # https://discuss.pytorch.org/t/how-can-l-load-my-best-model-as-a-feature-extractor-evaluator/17254/8

    # instead of forward in this way, you can also define it in the training batch as done for VAE in lightning
    # https://github.com/williamFalcon/pytorch-lightning-vae/blob/main/vae.py
    # for now we keep this simple
    
    # forward pass on a batch https://stackoverflow.com/questions/68507871/pytorch-custom-batch-function
    def forward(self, tab):
        intermediate_outputs = self.encode(tab)
        simulated_z = self.simulate_z(intermediate_outputs)
        # y_tilde = self.get_y_tilde(simulated_z)
        # p_prime_y_z = self.get_p_prime_y_z(y_tilde)
        # w = self.get_w(p_prime_y_z)
        return simulated_z
        # return intermediate_outputs, simulated_z, y_tilde, p_prime_y_z, w
    
    # on the difference between forward and training_step 
    # https://stackoverflow.com/questions/72437583/difference-between-forward-and-train-step-in-pytorch-lightning#:~:text=forward%3A%20Encapsulates%20the%20way%20the,%2C%20discriminators%2C%20loss%20functions%20etc.
    # change get_theta to loss
    # def training_step(self, batch, batch_idx):
    #     tabular, y = batch
    #     # forward_outcomes = self(tabular)
    #     intermediate_outputs, simulated_z, y_tilde, p_prime_y_z, w = self(tabular)
    #     # idea for this part taken from 
    #     # https://github.com/Lightning-AI/lightning-bolts/blob/master/pl_bolts/models/autoencoders/basic_vae/basic_vae_module.py
    #     loss, logs = self.get_theta_loss(intermediate_outputs, simulated_z, w)
    #     self.log_dict({f"train_{k}": v for k, v in logs.items()}, on_step=True, on_epoch=False)
    #     return loss
   
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr))
    # return {"loss": loss, "log": tensorboard_logs}

In [30]:
model = ModelCalculateZ(batch_size = 3)
my_model = model.forward(tabular)
my_model

# check if a pytorch function is differentiable (has a backward)
# https://discuss.pytorch.org/t/custom-loss-function-whats-legal-in-forward-pass/99817/4

# list(map(type, my_model))

# trainer = pl.Trainer()
# trainer.fit(model)

[[0,
  tensor([-0.6289, -0.2523, -0.9327,  2.2784, -0.8038, -0.8884, -2.1692,  0.6716,
           1.6416,  1.1684, -1.3054, -0.7240,  0.4866,  1.5288,  0.1526, -0.4053,
           1.5458,  1.1831,  0.4519, -0.8059]),
  tensor([ 0.0761, -0.8855, -0.8285,  0.2999,  0.2002, -2.4580, -1.3683,  0.3025,
           2.5616, -1.0606, -1.5528,  0.9380, -1.2108, -2.2475,  0.6879, -1.1863,
          -0.0834,  1.7049,  1.1421, -1.5640]),
  tensor([-2.2779,  1.0135, -0.3112,  0.9008,  0.8518, -2.0049, -2.3349, -0.7666,
           0.9505, -0.7515, -1.2388, -0.5911, -1.0143,  1.3951,  0.1658,  0.4330,
           1.2806,  1.4533,  1.4681, -0.5522]),
  tensor([-0.3548,  1.4702, -0.7649,  1.7090, -1.0846,  0.5053, -1.4858,  1.7263,
           0.9559, -2.4256, -0.8695, -1.6242,  1.0779, -0.7013, -0.1983, -2.5738,
           0.4856, -1.4274, -0.6856, -0.2395]),
  tensor([-0.2118,  0.5665, -1.9284,  0.9523, -0.9985, -2.0932,  0.6940,  0.8027,
           0.7751, -0.9680, -0.7892, -2.9849, -0.0099, -0.7307, -

In [ ]:
# check this out for `pdf` of a tensor
# https://deepmind.github.io/torch-distributions/#toc_0


In [ ]:
# load the data
train_batch_load = torch.utils.data.DataLoader(dataset=all_train_data, batch_size=3)
tabular, y = next(iter(train_batch_load))
print(tabular)
# check out this when you want to expand it to larger batches
# https://stackoverflow.com/questions/57386851/how-to-get-entire-dataset-from-dataloader-in-pytorch

In [10]:
%%time
# check this out https://discuss.pytorch.org/t/confusion-about-nested-modules-and-shared-parameters/26212/1
# https://discuss.pytorch.org/t/nested-modules-in-pytorch-and-the-parameters-update/70939
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        self.layer_h = nn.Linear(4, 10)
        self.layer_z = nn.Linear(10, 20)
        self.layer_j = nn.Linear(20, 10)
        self.layer_y = nn.Linear(10,1)
        
    def f(self, x):
        x = torch.sin(torch.exp(x)) + torch.log(x) - 1/x
        return x

    def simulate_z(self, z_tilde_outputs, R: int = 5):
        N = len(z_tilde_outputs)
        z_means = []
        # this N should probably by something else (automatically use the batch)
        for i in range(N):
            np.random.seed(1)
            # parallelize this over multiple CPUs
            # this basically means that the length of our list stays size as the batch size and then we next inside
            z_tilde_output = z_tilde_outputs[i]
            z_tilde_output = z_tilde_output.detach().numpy()
            z_means_2 = []
            for r in range(R):
                z_means_2.append(torch.from_numpy(np.random.normal(z_tilde_output)).float())
            z_means.append(z_means_2)
        return z_means
    # torch.stack(z_means, dim=1)
    
    # torch stack
    # https://discuss.pytorch.org/t/appending-to-a-tensor/2665/6
    def get_y_tilde(self, simulated_z):
        y_tilde = []
        for r in range(len(simulated_z)): #this was just an arbitary indexing since all Rs are the same
        # for copying the weights, you can also have a look at https://discuss.pytorch.org/t/loading-a-specific-layer-from-checkpoint/52725
        # we place 1 here in the second part since the second element of the list is the tensor (first is just the `ith` element)
            new_layer_j = self.layer_j(simulated_z[r]) #this is plus 1 since the first element is the index itself
            new_layer_y = self.layer_y(new_layer_j)
            y_tilde.append(new_layer_y)
        return torch.stack(y_tilde, dim=1)

    def get_p_prime_y_z(self, y_tilde):
        p_prime_y_z = []
        for r in range(len(y_tilde)):
            pdf_val = torch.from_numpy(scipy.stats.norm.pdf(y, y_tilde[r].detach().numpy(), 1))
            p_prime_y_z.append(pdf_val)
        return torch.stack(p_prime_y_z)

    def forward(self, x_batch):
        result = self.simulate_z(x_batch)
        # result_2 = self.get_y_tilde(result)
        # return result.size(),result
        # result_3 = self.get_p_prime_y_z(result_2)
        return result


torch.manual_seed(1)
net = NeuralNet()
net(my_model)


CPU times: user 3.62 ms, sys: 0 ns, total: 3.62 ms
Wall time: 11.1 ms


[[tensor([ 0.8978, -1.4379, -0.4041, -2.0714,  1.8633, -1.5536,  2.5196,  0.1981,
          -0.6785, -1.2434,  1.9419, -1.4192, -1.3151, -0.1647,  2.0957, -1.8361,
          -0.6611,  0.1221,  1.0369,  0.0139]),
  tensor([-1.8272,  0.3185,  1.0257, -0.4959,  1.8987,  0.0642,  0.6519,  0.0236,
          -1.2654, -0.4637, -0.2119,  0.2442, -1.6798, -0.6259,  0.2907, -0.7489,
          -1.6060,  1.2344,  2.6545,  0.1731]),
  tensor([-0.9184, -1.7138, -0.6230,  0.6941,  1.0487,  0.1110,  0.9657,  3.0596,
          -0.8774, -0.3768,  0.7799,  0.2887, -2.1352, -0.1300,  0.7530, -0.1496,
           0.3503,  1.9311,  1.2803,  0.3162]),
  tensor([-1.4810,  0.4267,  0.6370, -1.2965,  1.4864,  0.6724,  1.9065,  2.4791,
           1.1881, -2.3905, -0.9644,  0.1365, -0.8326,  1.0955,  1.2776, -2.7584,
          -0.7949,  1.8280,  1.2248,  0.1931]),
  tensor([-0.9489, -1.0270,  0.3107, -0.5883,  1.1962,  0.8670,  0.1042,  1.3369,
          -0.8757,  0.1355,  1.6787,  0.8261, -1.3679, -0.4194,  1.385

### Model done in Pytroch way (18th of October)

In [11]:
%%time
# take inspiration from here
# https://github.com/ethanluoyc/pytorch-vae/blob/master/vae.py

# check this out https://discuss.pytorch.org/t/confusion-about-nested-modules-and-shared-parameters/26212/1
# https://discuss.pytorch.org/t/nested-modules-in-pytorch-and-the-parameters-update/70939
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        
        self.layer_h = nn.Linear(4, 10)
        self.layer_z = nn.Linear(10, 20)
        self.layer_j = nn.Linear(20, 10)
        self.layer_y = nn.Linear(10,1)

    
    def simulate_z(self, z_tilde_output, R: int = 5):
        # this N should probably by something else (automatically use the batch)
        # parallelize this over multiple CPUs
        # this basically means that the length of our list stays size as the batch size and then we next inside
        np.random.seed(1)
        z_tilde_output = z_tilde_output.detach().numpy()
        # return z_tilde_output
        z_means = []
        for r in range(R):
            # please note that we are generating the `z` for all the batch here rather than `R` times `z` for each observation
            # this is an important observation
            z_means.append(torch.from_numpy(np.random.normal(z_tilde_output)).float())
        return torch.stack(z_means,dim=1)
        # this could be wrong, so double check
        
    # torch stack
    # https://discuss.pytorch.org/t/appending-to-a-tensor/2665/6
    def get_y_tilde(self, simulated_z):
        # y_tilde = []
        # for r in range(len(simulated_z)): #this was just an arbitary indexing since all Rs are the same
        # for copying the weights, you can also have a look at https://discuss.pytorch.org/t/loading-a-specific-layer-from-checkpoint/52725
        # we place 1 here in the second part since the second element of the list is the tensor (first is just the `ith` element)
        new_layer_j = self.layer_j(simulated_z) #this is plus 1 since the first element is the index itself
        new_layer_y = self.layer_y(new_layer_j)
        # y_tilde.append(new_layer_y)
        return new_layer_y

    # THIS IS WRONG
    def get_p_prime_y_z(self, y_tilde, y):
        p_prime_y_z = []
        for r in range(len(y_tilde)):
        #     pdf_val = torch.from_numpy(scipy.stats.norm.pdf(y, y_tilde[r].detach().numpy(), 1))            
            # doing it in a torch way as done here https://stackoverflow.com/questions/73634337/getting-the-probability-density-value-for-a-given-distribution-in-pytorch/73634537
            dist = torch.distributions.normal.Normal(r, 1)
            p_prime_y_z.append(torch.exp(dist.log_prob(y)))
        return torch.stack(p_prime_y_z, dim=1)
        # p_prime_y_z.append(pdf_val)
        # return torch.stack(p_prime_y_z, dim=1)

    def forward(self, x_batch, y):
        result = self.simulate_z(x_batch)
        result_2 = self.get_y_tilde(result)
        # return result.size(),result
        result_3 = self.get_p_prime_y_z(result_2, y)
        # return result_3
        return result_2.size(), result_2, result_3


torch.manual_seed(1)
net = NeuralNet()
net(my_model, y)



CPU times: user 3.54 ms, sys: 0 ns, total: 3.54 ms
Wall time: 6.13 ms


(torch.Size([3, 5, 1]),
 tensor([[[ 0.0229],
          [ 0.2109],
          [ 0.1282],
          [-0.1737],
          [ 0.2623]],
 
         [[-0.0660],
          [-0.0074],
          [ 0.6883],
          [-0.1972],
          [ 0.1826]],
 
         [[-0.0883],
          [ 0.2200],
          [ 0.2102],
          [ 1.0924],
          [ 0.1187]]], grad_fn=<AddBackward0>),
 tensor([[2.0864e-12, 1.7084e-09, 5.1462e-07],
         [3.0582e-12, 2.3743e-09, 6.7810e-07],
         [7.5534e-14, 9.6667e-11, 4.5511e-08]], dtype=torch.float64))

In [15]:
# Seed everything
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

logger = TensorBoardLogger("lightning_logs", name="tabular_vae")
early_stop_callback = EarlyStopping(monitor="val_rmse_loss", patience=20, verbose=True, mode="min", check_on_train_epoch_end = False)
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_rmse_loss", mode="min")

model = ModelCalculateZ(batch_size = 112)
trainer = pl.Trainer(accelerator='gpu', devices=1, logger=logger, max_epochs=350, callbacks=[early_stop_callback,checkpoint_callback],enable_checkpointing=True)

lr_finder = trainer.tuner.lr_find(model)
fig = lr_finder.plot(suggest=True, show=True)
new_lr = lr_finder.suggestion()
print("The best learning rate is", new_lr)
model.hparams.lr = new_lr
# model.hparams.lr = 0.0015

trainer.fit(model)

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


MisconfigurationException: No `training_step()` method defined. Lightning `Trainer` expects as minimum a `training_step()`, `train_dataloader()` and `configure_optimizers()` to be defined.

## chatGPT aid

In [21]:
import torch
import torch.nn as nn
import pytorch_lightning as pl

# Define your model class
class MultiModalModel(pl.LightningModule):
    def __init__(self, num_latent_vars, num_observed_vars):
        super(MultiModalModel, self).__init__()
        self.num_latent_vars = num_latent_vars
        self.num_observed_vars = num_observed_vars
        
        # Define the model parameters (e.g. weights and biases) here
        self.param1 = nn.Parameter(torch.randn(num_latent_vars, num_observed_vars))
        self.param2 = nn.Parameter(torch.randn(num_latent_vars))
        
        # Define a new attribute to store the expected log likelihood
        self.expected_log_likelihood = None
        
    def forward(self, x):
        # Define the forward pass of the model here
        return self.param1@x + self.param2
    
    def expectation_step(self, x):
        # Estimate the expectation of the latent variables given the current model parameters
        latent_mean = self.forward(x)
        latent_var = torch.ones(self.num_latent_vars)
        latent_dist = torch.distributions.Normal(latent_mean, latent_var)
        return latent_dist
    
    def maximization_step(self, x, latent_dist):
        # Update the model parameters by maximizing the expected log-likelihood
        self.expected_log_likelihood = latent_dist.log_prob(self.forward(x)).mean()
        self.expected_log_likelihood.backward(retain_graph=True)
        with torch.no_grad():
            self.param1 -= self.param1.grad
            self.param2 -= self.param2.grad
            self.zero_grad()
    
    def training_step(self, batch, batch_idx):
        # Perform the training step
        x, y = batch
        latent_dist = self.expectation_step(x)
        self.maximization_step(x, latent_dist)
        
        # Return the expected log likelihood as the loss
        return self.expected_log_likelihood
    
    def configure_optimizers(self):
        # Configure the optimizers
        return torch.optim.SGD(self.parameters(), lr=0.1)

# Create an instance of the model and some input data
model = MultiModalModel(num_latent_vars=5, num_observed_vars=10)
x = torch.randn(10, 1)
y = torch.randn(5, 1)

# Create a data loader
dataloader = torch.utils.data.DataLoader([(x, y)], batch_size=1)

# Create a Lightning trainer
trainer = pl.Trainer(max_epochs=100)

# Train the model
trainer.fit(model, dataloader)


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
------------------------------
55        Trainable params
0         Non-trainable params
55        Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 163.45it/s, loss=-0.919, v_num=7]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 40.71it/s, loss=-0.919, v_num=7] 


In [13]:
import torch
import torch.nn as nn

# Define your model class
class MultiModalModel(nn.Module):
    def __init__(self, num_latent_vars, num_observed_vars):
        super(MultiModalModel, self).__init__()
        self.num_latent_vars = num_latent_vars
        self.num_observed_vars = num_observed_vars
        
        # Define the model parameters (e.g. weights and biases) here
        self.param1 = nn.Parameter(torch.randn(num_latent_vars, num_observed_vars))
        self.param2 = nn.Parameter(torch.randn(num_latent_vars))
        
    def forward(self, x):
        # Define the forward pass of the model here
        return self.param1 @ x + self.param2
    
    def expectation_step(self, x):
        # Estimate the expectation of the latent variables given the current model parameters
        latent_mean = self.forward(x)
        latent_var = torch.ones(self.num_latent_vars)
        latent_dist = torch.distributions.Normal(latent_mean, latent_var)
        return latent_dist
    
    def maximization_step(self, x, latent_dist):
        # Update the model parameters by maximizing the expected log-likelihood
        expected_log_likelihood = latent_dist.log_prob(self.forward(x)).mean()
        expected_log_likelihood.backward()
        with torch.no_grad():
            self.param1 -= self.param1.grad
            self.param2 -= self.param2.grad
            self.zero_grad()

# Define the training loop
def train(model, x, num_iterations):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    for i in range(num_iterations):
        latent_dist = model.expectation_step(x)
        model.maximization_step(x, latent_dist)
        optimizer.step()

# Create an instance of the model and some input data
model = MultiModalModel(num_latent_vars=5, num_observed_vars=10)
x = torch.randn(10, 1)

# Train the model
train(model, x, num_iterations=10)


## Progress until here

In [ ]:
# for i in (simulated_z[1][1:]):
#     print(i.detach().numpy())
    
print(simulated_z[1][1:])
print("###")
print(intermediate_outputs['layer_z'][1])

# possibility one: 1 array size 20 x 1
# possibility two: 5 arrays size 1 x 1

In [ ]:
# step 11: optimize for theta
def get_thetas():
    theta_loss = []
    r_sum = []
    n_sum = []
    for i in range(len(w)):
        theta_loss.append([i])
        f_theta_l_xi = intermediate_outputs['layer_z'][i]
        w_multiplier =  w[i][1:]
        # I do w[1] since the rs are all equal (5) so it would be like 5r x 15i
        for r in range(len(w[1])-1):
            # # not sure about the calculation of the second component of mse so ask to marc-o about it
            # squared euclean norm here https://stats.stackexchange.com/questions/181620/what-is-the-meaning-of-super-script-2-subscript-2-within-the-context-of-norms
            mse = ((simulated_z[i][r+1] - f_theta_l_xi)** 2).detach().numpy()
            # # weighted_mse
            weighted_mse = np.sum(mse)
            # multiply and flatten numpy
            weighted_mse = np.ravel(np.multiply(weighted_mse,w_multiplier))
            
            theta_loss[i].append(weighted_mse)
        # theta_loss[i].append(r_sums)

        # sum the array by R
        r_sum.append(np.sum(theta_loss[i][1]))
        
    # sum by n
    n_sum = sum(r_sum)
    
    return n_sum

In [ ]:
###

In [ ]:
import torch
from torch.distributions.normal import Normal
import matplotlib.pyplot as plt

# learning_rate = 0.01
# for f in model.parameters():
#     f.data.sub_(f.grad.data * learning_rate)

    
"""
EM algo demo, in pytorch
"""

n = 40 # must be even number
k = 2
eps = torch.finfo(torch.float32).eps

d1 = Normal(-2.0, 0.5)
d2 = Normal(2.0, 0.5)
x1 = d1.sample((n//2,))
x2 = d2.sample((n//2,))
x = torch.cat((x1, x2)).view(-1, 1)

mu = torch.tensor([-3.0, -2.5])
stdev = torch.tensor([0.2, 0.2])
prior = torch.tensor([0.5, 0.5])
converged = False
i = 0

while not converged:
    prev_mu = mu.clone()
    prev_stdev = stdev.clone()

    h = Normal(mu, stdev)
    llhood = h.log_prob(x)
    weighted_llhood = llhood + prior.log()
    log_sum_lhood = torch.logsumexp(weighted_llhood, dim=1, keepdim=True)
    log_posterior = weighted_llhood - log_sum_lhood
    posterior = torch.exp(log_posterior)
    mu = torch.sum(posterior * x, dim=0) / (torch.sum(posterior, dim=0) + eps)
    variance = torch.sum(posterior * (x - mu) ** 2, dim=0) / (torch.sum(posterior, dim=0) + eps)
    stdev = variance.sqrt()
    prior = posterior.mean(0)

    converged = torch.allclose(mu, prev_mu) and torch.allclose(stdev, prev_stdev)

    i += 1

print(i , mu, stdev, posterior.mean(0))

In [ ]:
import torch

x = torch.randn(3)
print(x)
x.argmax(-1)

In [ ]:
# step 12: optimize for phi

In [ ]:
# step 13: assign the dags to the new parameters

In [ ]:
# Step 14: Go back to step 5 and repeat for another batch


In [ ]:
# https://discuss.pytorch.org/t/how-to-calculate-the-multivariate-normal-distribution-using-pytorch-and-math/105943

In [ ]:
# try this model with four inputs
